In [1]:
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import timeit

In [2]:
with open('low_var_port.pkl', 'rb') as f: d = pickle.load(f)

In [3]:
portfolio_weights = {'SPY':.5, 'TLT': .4, 'GLD': .1}

In [4]:

def allocate_shares_continuous(data, portfolio_weights,initial_money = 10000000):
    latest_prices = {}
    shares = {}
    date_start = data.index[0].date().isoformat()
    for k in portfolio_weights.keys():
        latest_prices[k] = data.loc[date_start][k]['prc']
        shares[k] = initial_money*portfolio_weights[k]/latest_prices[k]
    return shares, latest_prices, date_start

In [5]:
def initialize_df_continuous(data, portfolio_weights,initial_money=1000000):
    shares, latest_prices, date_start = allocate_shares_continuous(data, portfolio_weights,initial_money)
    ls_trades = []
    ls_tickers = []
    ls_labels = ['shr chg', 'prc', 'csh chg', 'tot shr', 'tot bal', 'divflag', 'rebalflag']
    for k in latest_prices.keys():
        ls_trades = ls_trades + [shares[k], latest_prices[k], -shares[k]*latest_prices[k], shares[k], shares[k]*latest_prices[k],0, 0]
        ls_tickers.append(k)
    ls_trades_iterables = [ls_tickers, ls_labels]
    trades_columns = pd.MultiIndex.from_product(ls_trades_iterables, names=['ticker', 'metric'])
    df_trades_initial = pd.DataFrame(np.array([ls_trades]),columns=trades_columns, index = [date_start])
    
    #Initialize df_cash dataframe
    df_cash_initial = pd.DataFrame(np.array([0]), columns = ['cash'], index = [date_start])
    
    #Initialize df_portfolio_detail dataframe
    ls_port_det_int_labels = ['price', 'return', 'weight', 'shares', 'value']
    ls_port_det_int_iterables = [ls_tickers, ls_port_det_int_labels]
    columns = pd.MultiIndex.from_product(ls_port_det_int_iterables, names=['ticker', 'metric'])
    df_portfolio_detail_initial = pd.DataFrame(columns=columns)

    #Initialize df_portfolio_sum dataframe
    ls_port_sum_labels = ['total_value', 'asset_value', 'cash', 'cash_pct']
    df_portfolio_sum_initial = pd.DataFrame(columns=ls_port_sum_labels)



    return df_trades_initial, df_cash_initial, df_portfolio_sum_initial, df_portfolio_detail_initial

In [289]:

#Ning's rewrite
df_trades, df_cash, df_portfolio_sum, df_portfolio_detail = initialize_df_continuous(d, portfolio_weights,initial_money= 1000000)
arr_data_columns = d.columns.values
arr_data = np.array(d)
ls_prc_index = [x for x, y in enumerate(arr_data_columns) if y[1] == 'prc']
ls_ret_index = [x for x, y in enumerate(arr_data_columns) if y[1] == 'ret']
ls_tickers = list(set([y[0] for x, y in enumerate(arr_data_columns)]))
arr_trades_columns = df_trades.columns.values
arr_trades = np.append([[0]], np.array(df_trades), axis = 1)
arr_cash = np.array(df_cash)
arr_port_det = np.zeros([d.shape[0], df_portfolio_detail.shape[1]]) #Need to change back to data
arr_port_sum = np.zeros([d.shape[0], df_portfolio_sum.shape[1]]) #Need to change back to data
arr_portfolio_weights = np.array([i for i in portfolio_weights.values()])
ls_trades_shr_index = [x+1 for x, y in enumerate(arr_trades_columns) if y[1] == 'tot shr']
ls_purchase_price_index = [x+1 for x, y in enumerate(arr_trades_columns) if y[1] == 'prc']
ls_purchase_price = np.array(arr_trades[-1][ls_purchase_price_index])
arr_yesterday_prices =  arr_data[0][ls_prc_index]

#This loop goes through the data set "d", which should be a daily time series of asset returns and prices.
#Returns should be broken into returns including dividends, returns without dividends, and dividend only returns. 
arr_cur_shrs = np.array(arr_trades[-1][ls_trades_shr_index])
cum_ret={}
for k in range(len(ls_ret_index)):
    cum_ret[k]=arr_data[:,ls_ret_index[k]].cumprod()
    
cum_ret_1= pd.DataFrame(data=cum_ret)
ls_purchase_price_1 = pd.DataFrame(data=ls_purchase_price[np.newaxis])
ls_purchase_price_2 = np.array(ls_purchase_price_1)
cum_ret_1.columns = ls_tickers
cum_ret_1.columns = [str(col) + '_cum_ret' for col in cum_ret_1.columns]
ls_purchase_price_1.columns = ls_tickers
ls_purchase_price_1.columns = [str(col) + '_pur_prc' for col in ls_purchase_price_1.columns]
num_rows, num_cols = arr_data.shape
ls_cum_ret_index = [x for x, y in enumerate(cum_ret_1)]
ls_cum_ret_index=[x+num_cols for x in ls_cum_ret_index]
cum_ret_2 = np.array(cum_ret_1)
arr_data_1 = np.concatenate([arr_data, cum_ret_2],axis=1)
#Since the cum_ret_2 keeps track of the cumulative returns and when we rebalance, the cumulative return
#needs to be reset to 1, so create an array to keep track the cumulative returns right before rebalancing,
#and this will be used to reset the cumulative returns.
cum_ret_tracking = cum_ret_2[1,:]
cum_ret_tracking[:]=1

count = 0
for row in arr_data_1:
        
    tradeflag = 0
    arr_divflag = np.zeros(len(ls_tickers))
    arr_div_cash = np.zeros(len(ls_tickers))
    arr_rebal = np.zeros(len(ls_tickers))
    arr_new_shares = np.zeros(len(ls_tickers))
    arr_leftover_cash = 0
    arr_latest_prices = row[ls_prc_index]
    arr_latest_ret = row[ls_cum_ret_index]
    
    total_value = np.matmul(np.array(arr_latest_prices),arr_cur_shrs)+arr_cash[-1]
    arr_asset_val = (arr_cur_shrs * arr_latest_prices)
    arr_actual_weights = arr_asset_val/total_value
    dev_weights = np.absolute((arr_actual_weights/arr_portfolio_weights)-1)
        
    if trading_day_counter % 63 == 0 or np.amax(dev_weights) > 1:
        tradeflag = 1
        total_value = np.sum(ls_purchase_price_1 *arr_cur_shrs *arr_latest_ret/cum_ret_tracking)+arr_cash[-1]
        cum_ret_tracking = arr_latest_ret
        ls_purchase_price_2 = arr_latest_prices
        arr_shares = (total_value*arr_portfolio_weights)/arr_latest_prices
        arr_trade_add = np.array([count])
        for i in range(len(ls_tickers)):
                arr_trade_add = np.append(arr_trade_add, np.array([arr_new_shares[i], arr_latest_prices[i], -arr_new_shares[i] * arr_latest_prices[i], arr_trades[-1][ls_trades_shr_index[i]]+arr_new_shares[i], (arr_trades[-1][ls_trades_shr_index[i]]+arr_new_shares[i])*arr_latest_prices[i], arr_divflag[i], tradeflag]))
        arr_trades = np.append(arr_trades,[arr_trade_add], axis = 0)
        arr_cash = np.append(arr_cash, [np.array(cash)], axis=0)
        trading_day_counter=1
        arr_cur_shrs = np.array(arr_trades[-1][ls_trades_shr_index])
        arr_new_port_det = np.array([], dtype='f8')
        arr_asset_val = (arr_cur_shrs * arr_latest_prices)     
    else:
            trading_day_counter += 1
        
        arr_yesterday_prices = arr_latest_prices    
            
        #Reaggregates all the data into the dataframe "portfolio_detail" and "portfolio_sum"
        
        arr_new_port_det = np.array([], dtype='f8')
        arr_asset_val = (arr_cur_shrs * arr_latest_prices)
        total_asset_value = np.sum(arr_asset_val)
        arr_actual_weights = arr_asset_val/total_value
        ls_new_port_det = []
        for i in range(len(ls_tickers)):
            
            ls_new_port_det = ls_new_port_det + [arr_latest_prices[i], row[ls_ret_index[i]], arr_actual_weights[i], arr_cur_shrs[i], arr_asset_val[i]]

        arr_port_det[count] = ls_new_port_det
        
        arr_new_port_sum = [total_value, total_asset_value, arr_cash[-1].item(), arr_cash[-1].item()/(total_value)]
        
        arr_port_sum[count] = arr_new_port_sum
        count += 1
    return arr_port_det, arr_port_sum, arr_trades, arr_cash

In [295]:
ls_purchase_price_1

,GLD_pur_prc,TLT_pur_prc,SPY_pur_prc
0,117.419998,88.260002,44.779999
